In [24]:
from collections import Counter
from itertools import chain
import json
import pickle
from typing import Tuple

from geonorm.geonormaliser_utils import decompose
import nltk
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy as sa
import tqdm

pd.set_option("display.max_columns", None)

In [47]:
CHUNKSIZE = 100

In [48]:
def preprocess_full_name(name: str) -> str:
    if pd.isna(name):
        return np.nan
    
    name = name.upper()
    tokens = nltk.word_tokenize(name)
    tokens = filter(lambda x: x not in stopwords, tokens)
    
    return " ".join(tokens)


def parse_address(addr: str) -> Tuple[str]:
    if pd.isna(addr):
        return np.nan, np.nan, np.nan
    
    parts = decompose(addr)
    
    return parts["region"], parts["settlement"], parts["street"]

In [49]:
base_df = pd.read_csv(
    "data/База по всем организациям и ИП.zip",
    sep=";", 
    chunksize=CHUNKSIZE, 
    dtype=str
)
with open("stopwords.json") as f:
    stopwords = json.load(f)

In [50]:
for chunk in tqdm.tqdm(base_df):
    chunk["name"] = chunk["Наименование полное"].apply(preprocess_full_name)
    #chunk["name_short"] = chunk["Наименование краткое"].str.upper()
    chunk["tax_number"] = chunk["ИНН"]
    chunk["individual"] = chunk["ОКОПФ (расшифровка)"] == "Индивидуальный предприниматель"
    chunk["creation_date"] = pd.to_datetime(chunk["Дата создания"], errors="coerce").dt.date
    chunk["active"] = chunk["Компания действующая (1) или нет (0)"].fillna(0).astype(bool)
    chunk["activity_code"] = chunk["ОКВЭД2"]
    chunk = chunk.melt(
        id_vars=["name", "tax_number", "individual", "creation_date", "active", "activity_code"],
        value_vars=["Юр адрес", "Факт адрес"],
        var_name="addr_type",
        value_name="address",
    )
    
    #chunk[["region", "settlement", "street"]] = chunk["address"].apply(parse_address)
    break

0it [00:00, ?it/s]


In [51]:
%%time
chunk["address"].apply(parse_address)

CPU times: user 33.9 s, sys: 11.9 ms, total: 33.9 s
Wall time: 33.9 s


0                (Амурская, Магдагачи, К.Маркса)
1          (Амурская, Свободный, 40 Лет Октября)
2           (Амурская, Новобурейский, Советская)
3           (Амурская, Благовещенск, Пионерская)
4           (Амурская, Февральск, Магистральная)
                         ...                    
195                   (Крым, Водное, Молодежная)
196                  (Крым, Симферополь, Гоголя)
197    (Крым, Симферополь, Богдана Хмельницкого)
198     (Крым, Симферополь, Александра Невского)
199                  (Крым, Симферополь, Чехова)
Name: address, Length: 200, dtype: object

In [52]:
chunk["address"].to_list()

['Амурская обл., Магдагачинский район, пгт. Магдагачи, ул. К.Маркса, д. 19 каб. 104',
 'Амурская обл., г. Свободный, ул. 40 Лет Октября, д. 92',
 'Амурская обл, м.о. Бурейский, пгт Новобурейский, ул Советская, д. 53',
 'Амурская обл., г. Благовещенск, ул. Пионерская, д. 1',
 'Амурская обл., Селемджинский район, пгт. Февральск, ул. Магистральная, д. 3 к. НЕТ кв. НЕТ',
 'Амурская обл., Сковородинский район, г. Сковородино, ул. 60 Лет Ссср, д. 2',
 'Амурская обл., г. Благовещенск, ул. Ленина, д. 154 к. А кв. 3',
 'Амурская обл., Свободненский район, пос. Подгорный, ул. Центральная, д. 45',
 'Амурская обл., Завитинский район, с. Успеновка, ул. Центральная, д. 40А',
 'Амурская обл., Зейский район, с. Сосновый Бор, ул. Новая, домовладение 2',
 'Амурская обл., г. Тында, ул. Красная Пресня, д. 47',
 'Амурская обл., Селемджинский район, пгт. Февральск, ул. Ленина, д. 12 офис 121',
 'Амурская обл., Селемджинский район, пгт. Экимчан, ул. Школьная, д. 19',
 'Амурская обл., Серышевский район, с. Бо